In [1]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
from nba_api.stats.endpoints import fantasywidget
fantasy = fantasywidget.FantasyWidget()
df = fantasy.get_data_frames()[0]

In [3]:
df.head()

,PLAYER_ID,PLAYER_NAME,PLAYER_POSITION,TEAM_ID,TEAM_ABBREVIATION,GP,MIN,FAN_DUEL_PTS,NBA_FANTASY_PTS,PTS,REB,AST,BLK,STL,TOV,FG3M,FGA,FG_PCT,FTA,FT_PCT
0,203999,Nikola Jokic,C,1610612743,DEN,58,35.4,53.7,55.8,26.4,11.0,8.7,0.66,1.41,3.1,1.4,18.3,0.568,5.0,0.854
1,203507,Giannis Antetokounmpo,F,1610612749,MIL,49,33.8,52.2,54.7,28.5,11.3,6.0,1.27,1.16,3.7,1.1,18.3,0.564,10.0,0.683
2,201935,James Harden,G,1610612751,BKN,42,37.1,50.8,52.7,25.2,8.0,10.9,0.69,1.19,4.1,2.8,17.2,0.463,7.5,0.870
3,1629029,Luka Doncic,F-G,1610612742,DAL,53,35.1,50.3,52.0,28.7,8.0,8.7,0.64,1.02,4.3,3.0,21.0,0.484,7.4,0.726
4,203954,Joel Embiid,C-F,1610612755,PHI,41,32.1,49.7,52.1,30.0,11.1,3.1,1.41,1.00,3.2,1.2,18.4,0.513,11.7,0.851


In [4]:
df.loc[(df.PLAYER_NAME == 'Stephen Curry')]

,PLAYER_ID,PLAYER_NAME,PLAYER_POSITION,TEAM_ID,TEAM_ABBREVIATION,GP,MIN,FAN_DUEL_PTS,NBA_FANTASY_PTS,PTS,REB,AST,BLK,STL,TOV,FG3M,FGA,FG_PCT,FTA,FT_PCT
7,201939,Stephen Curry,G,1610612744,GSW,51,34.0,45.9,47.3,31.1,5.5,5.9,0.12,1.2,3.3,5.1,21.0,0.486,6.0,0.922


In [7]:
# get teams data
from nba_api.stats.static import teams
df_teams = pd.DataFrame(teams.get_teams())
df_teams.rename(columns={'id': 'team_id'}, inplace=True)
df_teams.head()

,team_id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [28]:
merged = df.merge(df_teams[['abbreviation', 'full_name']], left_on='TEAM_ABBREVIATION', right_on='abbreviation', how='inner')
merged.rename(columns={'full_name': 'TEAM_NAME'}, inplace=True)

### Joining on another data set

In [45]:
merged = pd.read_csv('../data/player_stats.csv')
merged.head()

,PLAYER_ID,PLAYER_NAME,PLAYER_POSITION,TEAM_ID,TEAM_ABBREVIATION,GP,MIN,FAN_DUEL_PTS,NBA_FANTASY_PTS,PTS,...,BLK,STL,TOV,FG3M,FGA,FG_PCT,FTA,FT_PCT,abbreviation,TEAM_NAME
0,203999,Nikola Jokic,C,1610612743,DEN,58,35.4,53.7,55.8,26.4,...,0.66,1.41,3.1,1.4,18.3,0.568,5.0,0.854,DEN,Denver Nuggets
1,1627750,Jamal Murray,G,1610612743,DEN,48,35.5,34.2,35.8,21.2,...,0.27,1.33,2.3,2.7,16.5,0.477,3.2,0.869,DEN,Denver Nuggets
2,1629008,Michael Porter Jr.,F,1610612743,DEN,48,30.9,30.3,31.8,17.6,...,0.92,0.60,1.1,2.4,12.6,0.541,2.1,0.772,DEN,Denver Nuggets
3,203932,Aaron Gordon,F,1610612743,DEN,38,28.6,26.7,28.2,13.3,...,0.74,0.71,2.2,1.4,10.5,0.460,3.4,0.651,DEN,Denver Nuggets
4,203115,Will Barton,G,1610612743,DEN,55,31.5,23.7,25.0,12.9,...,0.42,0.91,1.7,1.8,11.1,0.426,2.2,0.785,DEN,Denver Nuggets


In [12]:
matchup = pd.read_csv('../data/2021_schedule.csv')

In [29]:
columns = ['PLAYER_ID', 'PLAYER_NAME', 'PLAYER_POSITION', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'GP', 'MIN', 'PTS', 'REB', 'AST', 'BLK', 'STL', 'TOV', 'FG3M', 'FGA', 'FG_PCT',
       'FTA', 'FT_PCT', 'TEAM_NAME']

In [46]:
matchup

,date,visitor,vistor_pts,home,home_pts,dow,week
0,12/22/2020,Golden State Warriors,99.0,Brooklyn Nets,125.0,Tue,Week 1
1,12/22/2020,Los Angeles Clippers,116.0,Los Angeles Lakers,109.0,Tue,Week 1
2,12/23/2020,Charlotte Hornets,114.0,Cleveland Cavaliers,121.0,Wed,Week 1
3,12/23/2020,New York Knicks,107.0,Indiana Pacers,121.0,Wed,Week 1
4,12/23/2020,Miami Heat,107.0,Orlando Magic,113.0,Wed,Week 1
...,...,...,...,...,...,...,...
1075,05/16/2021,Denver Nuggets,NaN,Portland Trail Blazers,NaN,Sun,Week 20
1076,05/16/2021,Utah Jazz,NaN,Sacramento Kings,NaN,Sun,Week 20
1077,05/16/2021,Phoenix Suns,NaN,San Antonio Spurs,NaN,Sun,Week 20
1078,05/16/2021,Indiana Pacers,NaN,Toronto Raptors,NaN,Sun,Week 20


In [31]:
temp = matchup.loc[(matchup.week == 'Week 17')]
temp

,date,visitor,vistor_pts,home,home_pts,dow,week
850,04/19/2021,Cleveland Cavaliers,105.0,Detroit Pistons,109.0,Mon,Week 17
851,04/19/2021,Chicago Bulls,102.0,Boston Celtics,96.0,Mon,Week 17
852,04/19/2021,Golden State Warriors,107.0,Philadelphia 76ers,96.0,Mon,Week 17
853,04/19/2021,San Antonio Spurs,109.0,Indiana Pacers,94.0,Mon,Week 17
854,04/19/2021,Houston Rockets,91.0,Miami Heat,113.0,Mon,Week 17
855,04/19/2021,Phoenix Suns,128.0,Milwaukee Bucks,127.0,Mon,Week 17
856,04/19/2021,Oklahoma City Thunder,107.0,Washington Wizards,119.0,Mon,Week 17
857,04/19/2021,Memphis Grizzlies,137.0,Denver Nuggets,139.0,Mon,Week 17
858,04/19/2021,Utah Jazz,111.0,Los Angeles Lakers,97.0,Mon,Week 17
859,04/20/2021,Orlando Magic,96.0,Atlanta Hawks,112.0,Tue,Week 17


In [32]:
t1 = temp[['visitor', 'home']]
t2 = temp[['home', 'visitor']].rename(columns={'home':'visitor', 'visitor':'home'})
test = pd.concat([t1,t2])
test

,visitor,home
850,Cleveland Cavaliers,Detroit Pistons
851,Chicago Bulls,Boston Celtics
852,Golden State Warriors,Philadelphia 76ers
853,San Antonio Spurs,Indiana Pacers
854,Houston Rockets,Miami Heat
...,...,...
899,Portland Trail Blazers,Memphis Grizzlies
900,Washington Wizards,Cleveland Cavaliers
901,Atlanta Hawks,Milwaukee Bucks
902,Orlando Magic,Indiana Pacers


In [38]:
player = merged.loc[(merged.PLAYER_NAME == 'Stephen Curry')]

In [37]:
opps_team = test.loc[(test.visitor == player.TEAM_NAME.values[0]), 'home'].values

In [47]:
opps_team

array(['Philadelphia 76ers', 'Washington Wizards', 'Denver Nuggets',
       'Sacramento Kings'], dtype=object)

In [39]:
player

,PLAYER_ID,PLAYER_NAME,PLAYER_POSITION,TEAM_ID,TEAM_ABBREVIATION,GP,MIN,FAN_DUEL_PTS,NBA_FANTASY_PTS,PTS,...,BLK,STL,TOV,FG3M,FGA,FG_PCT,FTA,FT_PCT,abbreviation,TEAM_NAME
131,201939,Stephen Curry,G,1610612744,GSW,51,34.0,45.9,47.3,31.1,...,0.12,1.2,3.3,5.1,21.0,0.486,6.0,0.922,GSW,Golden State Warriors


In [43]:
columns = ['PLAYER_NAME', 'PLAYER_POSITION', 'GP', 'MIN', 'PTS', 'REB', 'AST', 'BLK', 'STL', 'TOV', 'FG3M', 'FGA', 'FG_PCT',
       'FTA', 'FT_PCT', 'TEAM_NAME']

In [44]:
merged.loc[(merged.PLAYER_POSITION.str.contains(player.PLAYER_POSITION.values[0])) &
           (merged.TEAM_NAME.isin(opps_team)) &
           (merged.MIN > 10), columns]

,PLAYER_NAME,PLAYER_POSITION,GP,MIN,PTS,REB,AST,BLK,STL,TOV,FG3M,FGA,FG_PCT,FTA,FT_PCT,TEAM_NAME
1,Jamal Murray,G,48,35.5,21.2,4.0,4.8,0.27,1.33,2.3,2.7,16.5,0.477,3.2,0.869,Denver Nuggets
4,Will Barton,G,55,31.5,12.9,4.1,3.3,0.42,0.91,1.7,1.8,11.1,0.426,2.2,0.785,Denver Nuggets
6,Monte Morris,G,44,26.1,10.3,2.2,3.2,0.30,0.70,0.7,1.1,8.3,0.477,1.6,0.789,Denver Nuggets
9,PJ Dozier,G-F,43,20.7,7.1,3.7,1.7,0.42,0.60,0.9,0.9,6.5,0.412,1.4,0.639,Denver Nuggets
10,Austin Rivers,G,22,20.7,7.2,2.3,1.9,0.00,0.55,1.0,1.3,6.2,0.431,0.7,0.750,Denver Nuggets
11,Facundo Campazzo,G,52,19.4,5.2,1.7,2.8,0.21,1.00,1.0,1.2,4.2,0.388,1.0,0.824,Denver Nuggets
76,Ben Simmons,G-F,48,33.3,14.8,7.6,7.1,0.65,1.60,3.3,0.1,10.4,0.545,5.5,0.615,Philadelphia 76ers
77,Shake Milton,G-F,51,23.9,13.4,2.3,3.0,0.25,0.63,1.8,1.1,10.9,0.450,2.9,0.849,Philadelphia 76ers
78,Danny Green,G,58,28.4,9.7,3.7,1.7,0.86,1.31,1.0,2.6,7.9,0.422,0.6,0.758,Philadelphia 76ers
79,Seth Curry,G,46,29.0,12.1,2.1,2.8,0.17,0.78,1.2,2.0,9.8,0.447,1.4,0.894,Philadelphia 76ers
